# Installazione dipendenze necessarie per Concorde e import librerie

In [0]:
! git clone https://github.com/jvkersch/pyconcorde

In [0]:
cd pyconcorde

In [0]:
! pip install -e .

In [0]:
pip install pyGPGO

In [0]:
from concorde.tsp import TSPSolver
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from tqdm import tqdm
from scipy.spatial import distance
import math
import operator
import time
from datetime import datetime
from tqdm import tqdm
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import collections
from yellowbrick.cluster import KElbowVisualizer
from scipy.spatial import distance_matrix
import itertools
import requests

from pyGPGO.covfunc import squaredExponential
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.surrogates.RandomForest import RandomForest
from pyGPGO.GPGO import GPGO
from pyGPGO.acquisition import Acquisition

# Import Dati

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
data = pd.read_csv("/content/drive/My Drive/Decision_Models/cities.csv")

# Funzioni utili

## Calcolo numeri primi

In [0]:
def sieve_of_eratosthenes(n):
    primes = [True for i in range(n+1)]
    primes[0] = False
    primes[1] = False
    for i in range(2,int(np.sqrt(n)) + 1):
        if primes[i]:
            k = 2
            while i*k <= n:
                primes[i*k] = False
                k += 1
    return(primes)
prime_cities = sieve_of_eratosthenes(max(data["CityId"]))

## Calcolo distanza

In [0]:
def total_distance_v2(path):
    prev_city = path[0]
    total_distance = 0
    step_num = 1
    for city_num in path[1:]:
        next_city = city_num
        total_distance = total_distance + \
            np.sqrt(pow((data["X"][city_num] - data["X"][prev_city]),2) + pow((data["Y"][city_num] - data["Y"][prev_city]),2)) * \
            (1+ 0.1*((step_num % 10 == 0)*int(not(prime_cities[prev_city]))))
        prev_city = next_city
        step_num = step_num + 1
    return total_distance

## Nearest neighbor

In [0]:
def nearest_neighbour_v2(cities):
    ids = cities.CityId.values[1:]
    xy = np.array([cities.X.values, cities.Y.values]).T[1:]
    prima = cities.CityId.values[0]
    path = [prima, ]
    while len(ids) > 0:
        last_x, last_y = cities.X[path[-1]], cities.Y[path[-1]]
        dist = ((xy - np.array([last_x, last_y]))**2).sum(-1)
        nearest_index = dist.argmin()
        path.append(ids[nearest_index])
        ids = np.delete(ids, nearest_index, axis=0)
        xy = np.delete(xy, nearest_index, axis=0)
    path.append(prima)
    return path

## Ottimizzatore Prime Swap

In [0]:
def prime_swap(sequenza):
  nnpath_with_primes = sequenza.copy()
  for index in range(20,len(nnpath_with_primes)-30):
      city = nnpath_with_primes[index]
      if (prime_cities[city] &  ((index+1) % 10 != 0)):        
          for i in range(-1,4):
              tmp_path = nnpath_with_primes.copy()
              swap_index = (int((index+1)/10) + i)*10 - 1
              tmp_path[swap_index],tmp_path[index] = tmp_path[index],tmp_path[swap_index]
              if total_distance_v2(tmp_path[min(swap_index,index) - 1 : max(swap_index,index) + 2]) < total_distance_v2(nnpath_with_primes[min(swap_index,index) - 1 : max(swap_index,index) + 2]):
                  nnpath_with_primes = tmp_path.copy() 
                  break
  return nnpath_with_primes

## Merge cluster 

In [0]:
sequenza = []
def mergeclustersolver(cl1, cl2, solver_type, data, sequenza = sequenza):
  possible_solver = ["concorde", "nearest_neighbour"]
  primo = data[data["Cluster"] == cl1]
  secondo = data[data["Cluster"] == cl2]
  c1 = []
  c2 = []
  for i in primo.index:
    c1.append(list(data[data["Cluster"] == cl1][["X", "Y"]].loc[i]))
  for i in secondo.index:
    c2.append(list(data[data["Cluster"] == cl2][["X", "Y"]].loc[i]))
  distmat = (distance_matrix(c1, c2))
  #Le due città per cui far passare il segmento di unione dei cluster sono specificate da (riga, colonna) nella svaribile posizione
  posizione = np.argwhere(distmat==np.min(distmat))[0][0], np.argwhere(distmat==np.min(distmat))[0][1]
  #Ottenimento degli identificativi delle due città
  citta1 = data[data["Cluster"] == cl1].iloc[posizione[0]:posizione[0]+1]["CityId"].values[0]
  citta2 = data[data["Cluster"] == cl2].iloc[posizione[1]:posizione[1]+1]["CityId"].values[0]
  #cambio ordine dataframe
  prelevo = secondo[secondo["CityId"] == citta2]
  resto = secondo[secondo["CityId"] != citta2]
  ris = pd.concat([prelevo, resto], ignore_index=True)
  ris.index = ris.CityId.values
  if(solver_type in possible_solver):
    if(solver_type == "nearest_neighbour"):
      #applico nearest neighbour al risultato
      seq2 = nearest_neighbour_v2(ris)
      seq2.pop(-1)

      sequenza = sequenza[:sequenza.index(citta1)+1] + seq2 + sequenza[sequenza.index(citta1)+1:]
    
    elif(solver_type == "concorde"):
      #applico concorde al risultato
      solver = TSPSolver.from_data(
      ris.X * 1000,
      ris.Y * 1000,
      norm="EUC_2D")
      tour_data = solver.solve(time_bound = 60.0, verbose = True, random_seed = 2019)
      seq2 = tour_data.tour
      seq2 = list(ris.iloc[seq2].CityId.values)
      #unione dei cluster tramite la sequenza
      sequenza = sequenza[:sequenza.index(citta1)+1] + seq2 + sequenza[sequenza.index(citta1)+1:]

    return(sequenza)
    
  else:
    print("Solutore sbagliato!")

# Risoluzione problema completo con NN e concorde

In [0]:
sequenza_nn = nearest_neighbour_v2(data)
total_distance_v2(sequenza_nn)

In [0]:
total_distance_v2(prime_swap(sequenza_nn))

In [0]:
solver = TSPSolver.from_data(
    data.X * 1000,
    data.Y * 1000,
    norm="EUC_2D")
tour_data = solver.solve(time_bound = 60.0, verbose = True, random_seed = 2019)
sequenza_conc = tour_data.tour
sequenza_conc = np.append(sequenza_conc, [0])

total_distance_v2(sequenza_conc)

In [0]:
total_distance_v2(prime_swap(sequenza_conc))

# AutoML

In [0]:
def final_solver(k):
  #Ottengo i cluster
  mclusterer = GaussianMixture(n_components=int(k), tol=0.01, random_state=66, verbose=1)
  cluster_label = mclusterer.fit_predict(data[['X','Y']])
  data["Cluster"] = cluster_label

  #Calcolo i centroidi
  ncluster = len(data['Cluster'].unique())
  centroidi = pd.DataFrame(columns=['Cluster','X', "Y"])

  for x in range(ncluster):
    cent_X = data[data["Cluster"] == x]["X"].mean()
    cent_Y = data[data["Cluster"] == x]["Y"].mean()
    centroidi = centroidi.append({'Cluster': int(x),
                                  'X': cent_X,
                                  'Y': cent_Y},
                                ignore_index=True)

  #Applicazione concorde ai centroidi
  centroidi.columns=['CityId','X','Y']
  solver = TSPSolver.from_data(
      centroidi.X * 1000,
      centroidi.Y * 1000,
      norm="EUC_2D"
  )
  tour_data = solver.solve(time_bound = 60.0, verbose = True, random_seed = 2019) # solve() doesn't seem to respect time_bound for certain values?
  path_concorde = tour_data.tour
  seqcentr2 = list(centroidi.iloc[path_concorde].CityId.values) + [0]

  #Cluster iniziale
  cliniziale = data[data["CityId"]==0]["Cluster"][0]
  cl1 = data[data["Cluster"]==cliniziale]

  #Sistemazione sequenza
  seqcentr2.pop(-1)
  print(seqcentr2)
  start = seqcentr2.index(cliniziale)
  pre = seqcentr2[:start]
  post = seqcentr2[start:]
  seqcentr2 = post + pre
  print(seqcentr2)

  #Applicazione concorde su primo cluster
  solver = TSPSolver.from_data(
    cl1.X * 1000,
    cl1.Y * 1000,
    norm="EUC_2D")
  tour_data = solver.solve(time_bound = 60.0, verbose = True, random_seed = 2019)
  sequenza = tour_data.tour
  sequenza = list(cl1[cl1["Cluster"]==cliniziale].iloc[sequenza].CityId.values) + [0]

  #Applicazione concorde per tutti gli altri clsuter
  for i in tqdm(range(len(seqcentr2[:-1]))):
    sequenza = mergeclustersolver(seqcentr2[i], seqcentr2[i+1], "concorde", data, sequenza)
  sequenza_conccentr_concclust = sequenza
  sequenza_conccentr_concclust = prime_swap(sequenza_conccentr_concclust)
  distanza = total_distance_v2(sequenza_conccentr_concclust)
  return 1/distanza

In [0]:
range_cluster = [5, 2000]
gp_modelsur = RandomForest()
fz_acquisition1=Acquisition(mode="ExpectedImprovement")

param ={'k': ('int', range_cluster)}

In [0]:
def resultDataframe(k,dist):
  result = pd.DataFrame(k, columns=['k'])
  result['distance'] = 1/dist
  result['phase']="increment"
  result.loc[0:2,'phase']='initial'
  return result

In [0]:
np.random.seed(2019)

SMBO_ei = GPGO(gp_modelsur,fz_acquisition1,final_solver,param)
SMBO_ei.run(init_evals = 5, max_iter = 15)

In [0]:
risultati = resultDataFrame(SMBO_ei.GP.X, SMBO_ei.GP.y)

In [0]:
risultati

# Metodo cluster con k specifico

Da qui in poi il codice è da eseguire solo se si è interessati ad uno specifico numero di cluster k

## Calcolo Cluster

In [0]:
mclusterer = GaussianMixture(n_components=2000, tol=0.01, random_state=66, verbose=1)
cluster_label = mclusterer.fit_predict(data[['X','Y']])

In [0]:
data["Cluster"] = cluster_label

In [0]:
data.head()

### Calcolo centroidi cluster

In [0]:
ncluster = len(data['Cluster'].unique())
centroidi = pd.DataFrame(columns=['Cluster','X', "Y"])

for x in range(ncluster):
  cent_X = data[data["Cluster"] == x]["X"].mean()
  cent_Y = data[data["Cluster"] == x]["Y"].mean()
  centroidi = centroidi.append({'Cluster': int(x),
                                'X': cent_X,
                                'Y': cent_Y},
                               ignore_index=True)
  


In [0]:
centroidi.head()

In [0]:
# Visualizzazione

fig = plt.figure(figsize=(10,8))
plt.scatter(centroidi['X'], centroidi['Y'], marker = "o")

##Applicazione del Nearest Neighbor ai centroidi
L'obiettivo è quello di determinare l'ordine con cui affrontare il merge delle sequenze

In [0]:
centroidi.columns=['CityId','X','Y']
seqcentr = nearest_neighbour_v2(centroidi)

In [0]:
df_path = pd.DataFrame({'CityId':seqcentr}).merge(centroidi,how = 'left')
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(df_path['X'], df_path['Y'], marker = "o")
plt.show()

###Sistemazione della sequenza di centroidi dopo il NN:

In [0]:
cliniziale = data[data["CityId"]==0]["Cluster"][0]   #Individuazione cluster in cui è presente il polo nord
cl1 = data[data["Cluster"]==cliniziale]

In [0]:
seqcentr.pop(-1)
print(seqcentr)
start = seqcentr.index(cliniziale)
pre = seqcentr[:start]
post = seqcentr[start:]
seqcentr = post + pre
print(seqcentr)

### Individuazione path subottimale per ogni cluster con NN e unione

In [0]:
sequenza = nearest_neighbour_v2(cl1) # Sequenza di partenza

In [0]:
for i in tqdm(range(len(seqcentr[:-1]))):
  sequenza = mergeclustersolver(seqcentr[i], seqcentr[i+1], "nearest_neighbour", data, sequenza)

sequenza_nncentr_nnclust = sequenza

In [0]:
total_distance_v2(sequenza_nncentr_nnclust)

### Individuazione path subottimale per ogni cluster con concorde e unione

In [0]:
solver = TSPSolver.from_data(
    cl1.X * 1000,
    cl1.Y * 1000,
    norm="EUC_2D")
tour_data = solver.solve(time_bound = 60.0, verbose = True, random_seed = 2019)
sequenza = tour_data.tour
sequenza = list(cl1[cl1["Cluster"]==cliniziale].iloc[sequenza].CityId.values) + [0]

In [0]:
for i in tqdm(range(len(seqcentr[:-1]))):
  sequenza = mergeclustersolver(seqcentr[i], seqcentr[i+1], "concorde", data, sequenza)

sequenza_nncentr_concclust = sequenza

In [0]:
total_distance_v2(sequenza_nncentr_concclust)

##Applicazione del Concorde ai centroidi
L'obiettivo è quello di determinare l'ordine con cui affrontare il merge delle sequenze

In [0]:
centroidi.columns=['CityId','X','Y']
solver = TSPSolver.from_data(
    centroidi.X * 1000,
    centroidi.Y * 1000,
    norm="EUC_2D"
)

In [0]:
tour_data = solver.solve(time_bound = 60.0, verbose = True, random_seed = 2019) # solve() doesn't seem to respect time_bound for certain values?
path_concorde = tour_data.tour

In [0]:
seqcentr2 = list(centroidi.iloc[path_concorde].CityId.values) + [0]

In [0]:
df_path = pd.DataFrame({'CityId':seqcentr2}).merge(centroidi,how = 'left')
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(df_path['X'], df_path['Y'], marker = "o")

###Sistemazione della sequenza di centroidi dopo Concorde:

In [0]:
cliniziale = data[data["CityId"]==0]["Cluster"][0]
cl1 = data[data["Cluster"]==cliniziale]

In [0]:
seqcentr2.pop(-1)
print(seqcentr2)
start = seqcentr2.index(cliniziale)
pre = seqcentr2[:start]
post = seqcentr2[start:]
seqcentr2 = post + pre
print(seqcentr2)

### Individuazione path subottimale per ogni cluster con NN e unione

In [0]:
sequenza = nearest_neighbour_v2(cl1)

In [0]:
for i in tqdm(range(len(seqcentr2[:-1]))):
  sequenza = mergeclustersolver(seqcentr2[i], seqcentr2[i+1], "nearest_neighbour", data, sequenza)

sequenza_conccentr_nnclust = sequenza

In [0]:
total_distance_v2(sequenza_conccentr_nnclust)

### Individuazione path subottimale per ogni cluster con concorde e unione

In [0]:
solver = TSPSolver.from_data(
    cl1.X * 1000,
    cl1.Y * 1000,
    norm="EUC_2D")
tour_data = solver.solve(time_bound = 60.0, verbose = True, random_seed = 2019)
sequenza = tour_data.tour
sequenza = list(cl1[cl1["Cluster"]==cliniziale].iloc[sequenza].CityId.values) + [0]

In [0]:
for i in tqdm(range(len(seqcentr2[:-1]))):
  sequenza = mergeclustersolver(seqcentr2[i], seqcentr2[i+1], "concorde", data, sequenza)

sequenza_conccentr_concclust = sequenza

In [0]:
total_distance_v2(sequenza_conccentr_concclust)

In [0]:
df_path = pd.DataFrame({'CityId':sequenza_conccentr_concclust}).merge(data,how = 'left')
fig, ax = plt.subplots(figsize=(20,16))
ax.plot(df_path['X'], df_path['Y'], marker = "o", markersize = 5)
plt.show()

In [0]:
total_distance_v2(prime_swap(sequenza_conccentr_concclust))